In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from gurobipy import quicksum
from gurobipy import GRB,Model
import math

In [2]:
from scipy.stats import uniform


In [3]:

def distancepolar(r1, r2, theta1, theta2,calc):
    return np.sqrt(r1**2 + r2**2 - 2*r1*r2*calc)

In [4]:
def calculateDistance(x1,y1,x2,y2):
    dist = ((x2 - x1)*(x2 - x1) + (y2 - y1)*(y2-y1))
    return dist

In [5]:
# Create the model
m = Model('Baseball')

# Set parameters
m.setParam('OutputFlag', True)


Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-20


In [6]:
V = [1 for i in range(4)]
a = 5  #Radius Length to First Base
b = 10
y = 25 #Whole Radius
B =[(a,0),(a,a),(0,a)]
P = [1,.5,.5,.5]
Alpha = 10 
Beta = 5
Gamma = [2.5,2.5,2.5]
H = 4

In [7]:
n = 10000
start = 0
width = 25
R_data_uniform = uniform.rvs(size=n, loc = start, scale=width)
n = 10000
start = 0
width = (math.pi)/2
Theta_data_uniform = uniform.rvs(size=n, loc = start, scale=width)

pos = [(R_data_uniform[i]*math.cos(Theta_data_uniform[i]),R_data_uniform[i]*math.sin(Theta_data_uniform[i])) for i in range(n)]



In [8]:
pos[5][0]

0.2927106942331376

In [9]:
#Add Variables
X = m.addVars(3,name ='X')
Y = m.addVars(3,name = 'Y')
X4 = m.addVar((1),name = 'Player-4 X')
Y4 = m.addVar((1),name = 'P4 Y')
DT1 = m.addVars(n,name = 'X1Time from Distance to Ball')
DT2 = m.addVars(n,name = 'Time from Distance to Ball')
DT3 = m.addVars(n,name = 'Time from Distance to Ball')
DT4 = m.addVars(n,name = 'Time from Distance to Ball')
TM = m.addVars(n,name = 'Minimum Time')
#TS1 = m.addVars(n, name = 'Throw Time to Each Base 1')
#TS2 = m.addVars(n, name = 'Throw Time to Each Base 2')
#TS3 = m.addVars(n, name = 'Throw Time to Each Base 3')
#TS4 = m.addVars(n, name = 'Throw Time to Each Base 4')
#calc = m.addVars(n,name='temp')
#AvgHT1 = m.addVars((1),name = 'Average Throw Time 1')
#AvgHT2 = m.addVars((1),name = 'Average Throw Time 1')
#AvgHT3 = m.addVars((1),name = 'Average Throw Time 1')
#AvgHT4 = m.addVars((1),name = 'Average Throw Time 1')

In [10]:
#Add Constraints
m.addConstrs((calculateDistance(X[i],0,Y[i],0) >= b for i in range(3)), name = 'Outfield min constr')
m.addConstrs((calculateDistance(X[i],0,Y[i],0) <= y for i in range(3)), name = 'Outfield max constr')
m.addConstr(calculateDistance(X4,0,Y4,0) <= y, name = 'Infield max constr')


<gurobi.QConstr Not Yet Added>

In [11]:
m.addConstrs(DT1[i] == calculateDistance(X[0],pos[i][0],pos[i][1],Y[0])/V[0] for i in range(n))
m.addConstrs(DT2[i] == calculateDistance(X[1],pos[i][0],pos[i][1],Y[1])/V[0] for i in range(n))
m.addConstrs(DT3[i] == calculateDistance(X[2],pos[i][0],pos[i][1],Y[2])/V[0] for i in range(n))
m.addConstrs(DT4[i] == calculateDistance(X4,pos[i][0],pos[i][1],Y4)/V[0] for i in range(n))
m.addConstrs((TM[i] == gp.min_([DT1[i],DT2[i],DT3[i],DT4[i]]) for i in range(n)),"min_constraints")

{0: <gurobi.GenConstr *Awaiting Model Update*>,
 1: <gurobi.GenConstr *Awaiting Model Update*>,
 2: <gurobi.GenConstr *Awaiting Model Update*>,
 3: <gurobi.GenConstr *Awaiting Model Update*>,
 4: <gurobi.GenConstr *Awaiting Model Update*>,
 5: <gurobi.GenConstr *Awaiting Model Update*>,
 6: <gurobi.GenConstr *Awaiting Model Update*>,
 7: <gurobi.GenConstr *Awaiting Model Update*>,
 8: <gurobi.GenConstr *Awaiting Model Update*>,
 9: <gurobi.GenConstr *Awaiting Model Update*>,
 10: <gurobi.GenConstr *Awaiting Model Update*>,
 11: <gurobi.GenConstr *Awaiting Model Update*>,
 12: <gurobi.GenConstr *Awaiting Model Update*>,
 13: <gurobi.GenConstr *Awaiting Model Update*>,
 14: <gurobi.GenConstr *Awaiting Model Update*>,
 15: <gurobi.GenConstr *Awaiting Model Update*>,
 16: <gurobi.GenConstr *Awaiting Model Update*>,
 17: <gurobi.GenConstr *Awaiting Model Update*>,
 18: <gurobi.GenConstr *Awaiting Model Update*>,
 19: <gurobi.GenConstr *Awaiting Model Update*>,
 20: <gurobi.GenConstr *Awaiti

In [12]:
m.update()
m.Params.NonConvex = 2

Set parameter NonConvex to value 2


In [ ]:
m.setObjective(gp.quicksum(TM[i] for i in range(n)),GRB.MINIMIZE)
m.optimize()

status_code = {1:'LOADED', 2:'OPTIMAL', 3:'INFEASIBLE', 4:'INF_OR_UNBD', 5:'UNBOUNDED'}
status = m.status

print('The optimization status is {}' .format(status_code[status]))

m.printStats()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 50008 columns and 0 nonzeros
Model fingerprint: 0x620dc85d
Model has 40007 quadratic constraints
Model has 10000 general constraints
Variable types: 50008 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [7e-05, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e-08, 6e+02]
Presolve added 50000 rows and 40004 columns
Presolve time: 0.55s
Presolved: 50037 rows, 90024 columns, 240072 nonzeros
Presolved model has 40000 SOS constraint(s)
Presolved model has 4 quadratic constraint(s)
Presolved model has 11 bilinear constraint(s)
Variable types: 50024 continuous, 40000 integer (40000 binary)

Deterministic concurrent LP optimizer: primal and dual simplex 

 21655 20114 57341.2958 1887 9763 864730.595    0.00000   100%  29.0 1473s
 22137 20797 57598.4143 1912 9762 864730.595    0.00000   100%  29.0 1500s
 22820 21526 57598.4143 1934 9762 864730.595    0.00000   100%  29.0 1529s
 23243 21526  196.88317  863 10000 864730.595    0.00000   100%  29.0 1530s
 23549 22236 57598.4143 1956 9762 864730.595    0.00000   100%  28.8 1558s
 24259 23014 57598.4143 1980 9762 864730.595    0.00000   100%  28.8 1587s
 25037 23020 57598.4143 2006 9762 864730.595    0.00000   100%  28.6 1652s
H25040 23020                    864730.59484    0.00000   100%  28.5 1652s
 25043 23295 58757.9938 2007 9761 864730.595    0.00000   100%  28.5 1685s
 25318 24188 58732.1438 2011 9761 864730.595    0.00000   100%  28.5 1716s
H25933 24188                    864730.58896    0.00000   100%  28.3 1716s
 26211 25058 58539.9367 2043 9758 864730.589    0.00000   100%  28.2 1744s
 27081 25561 58995.9703 2079 9756 864730.589    0.00000   100%  28.0 1780s
 27584 26548 58995.9703 

 89559 88359 256014.115 7141 8449 864730.589    0.00000   100%  23.2 6522s
 90382 88739 258744.926 7222 8420 864730.589    0.00000   100%  23.1 8416s
 90762 89492 231277.274 7299 8387 864730.589    0.00000   100%  23.1 8459s
 91515 90238 233344.225 7382 8363 864730.589    0.00000   100%  23.0 8506s
 92261 91108 236359.254 7451 8334 864730.589    0.00000   100%  22.9 8555s
 93131 91817 240772.054 7522 8301 864730.589    0.00000   100%  22.9 8606s
 93840 92473 276844.268 7592 8278 864730.589    0.00000   100%  22.8 8654s
 94496 93051 247498.455 7659 8247 864730.589    0.00000   100%  22.8 8697s
 95074 93621 249294.713 7728 8224 864730.589    0.00000   100%  22.7 8738s
 95644 94237 250396.119 7795 8203 864730.589    0.00000   100%  22.7 8779s
 96260 94826 286146.408 7867 8183 864730.589    0.00000   100%  22.7 8818s
 96849 95485 291070.774 7934 8154 864730.589    0.00000   100%  22.7 8856s
 97508 95641 258772.854 7995 8134 864730.589    0.00000   100%  22.6 9039s
 97664 96311 293763.306 7

 154178 152929 463192.451 13647 5800 864730.589    0.00000   100%  19.8 15842s
 154952 153680 464578.553 13704 5779 864730.589    0.00000   100%  19.8 15889s
 155703 154456 465665.036 13753 5760 864730.589    0.00000   100%  19.8 15931s
 156479 155187 467152.020 13814 5739 864730.589    0.00000   100%  19.7 16717s
 157210 155859 468571.709 13878 5717 864730.589    0.00000   100%  19.7 17835s
 157882 156622 469842.701 13935 5699 864730.589    0.00000   100%  19.7 17892s
 158645 157342 471232.345 13994 5677 864730.589    0.00000   100%  19.6 17945s
 159365 158140 472528.087 14049 5656 864730.589    0.00000   100%  19.6 17996s
 160163 158934 561717.587 14103 5638 864730.589    0.00000   100%  19.6 18046s
 160957 159260 475200.905 14169 5616 864730.589    0.00000   100%  19.5 18150s
 161283 160056 562908.599 14169 5617 864730.589    0.00000   100%  19.5 18197s
 162079 160883 564777.076 14224 5594 864730.589    0.00000   100%  19.5 18246s
 162906 161609 477251.439 14278 5559 864730.589    0